In [1]:
import pandas as pd
import numpy as np
import requests
import json
import pandas as pd
from itertools import tee

In [3]:
df = pd.read_csv("TIEMPOS_VIAJE_SEG_CLUSTERS_SEGURIDAD_BOMBEROS.csv", delimiter=",")

In [54]:
df

,Unnamed: 0,Unnamed: 0.1,x,y,index_vertice_cercano,LATITUD,LONGITUD,distancia_h,1,concat,2,3,4,5,6
0,0,0,4.124433,-73.627487,33483,4.124,-73.627545,0.048541,655,4.124%2C-73.6275454,"[696, 4964]","[863, 6133]","[901, 6494]","[462, 3238]","[446, 2893]"
1,1,1,4.118694,-73.563922,37514,4.119,-73.563647,0.045658,1244,4.119%2C-73.5636471,"[1460, 13029]","[1627, 14197]","[914, 8034]","[868, 7336]","[1175, 8475]"
2,2,2,4.075202,-73.672954,52077,4.075,-73.672891,0.023477,1186,4.075%2C-73.6728906,"[546, 4714]","[674, 5200]","[1707, 15415]","[1268, 12159]","[1323, 11463]"
3,3,3,4.153048,-73.599674,11110,4.153,-73.599651,0.005913,944,4.153%2C-73.5996511,"[1425, 9692]","[1592, 10861]","[611, 2419]","[824, 4780]","[1109, 5697]"
4,4,4,4.148361,-73.613292,14637,4.148,-73.613417,0.042406,642,4.148%2C-73.6134169,"[1123, 8051]","[1291, 9219]","[318, 1032]","[692, 4531]","[808, 4055]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,195,4.158273,-73.651119,7341,4.158,-73.651163,0.030774,610,4.158%2C-73.6511635,"[1069, 8363]","[1237, 9532]","[928, 5738]","[1086, 8055]","[866, 5919]"
196,196,196,4.103017,-73.659508,45497,4.103,-73.659841,0.036985,775,4.103%2C-73.6598412,"[132, 767]","[440, 2307]","[1293, 11468]","[854, 8212]","[909, 7516]"
197,197,197,4.155809,-73.632643,8546,4.156,-73.632766,0.025231,337,4.156%2C-73.632766,"[881, 7051]","[1048, 8219]","[668, 3496]","[888, 6707]","[668, 4571]"
198,198,198,4.136147,-73.609554,23612,4.136,-73.609601,0.017196,758,4.136%2C-73.609601,"[1077, 8274]","[1245, 9442]","[348, 1158]","[485, 2581]","[526, 2686]"


In [35]:
df_nuevos_bomberos = pd.read_csv("Nuevos_bomberos.csv", delimiter=";")

In [39]:
df_nuevos_bomberos

,id,Name,Latitud,Longitud,concat
0,14,Equipamiento potencial 1,4.107019,-73.655174,4.107019258%2C-73.65517375
1,15,Equipamiento potencial 2,4.101542,-73.650027,4.10154238%2C-73.6500275
2,16,Equipamiento potencial 3,4.143916,-73.610024,4.143916394%2C-73.61002435
3,17,Equipamiento potencial 4,4.129254,-73.606926,4.129253505%2C-73.6069264
4,18,Equipamiento potencial 5,4.126492,-73.619860,4.126492293%2C-73.61986041


In [1]:
#Se quita por motivos de seguridad
key_API_google=""

In [10]:
df["concat"]=df['LATITUD'].astype(str)+"%2C"+df['LONGITUD'].astype(str)

In [11]:
destinos= np.array(df["concat"].to_list())

In [37]:
df_nuevos_bomberos["concat"]=df_nuevos_bomberos['Latitud'].astype(str)+"%2C"+df_nuevos_bomberos['Longitud'].astype(str)

In [38]:
origenes=np.array(df_nuevos_bomberos["concat"].to_list())

In [40]:
todas_solicitudes=[]
for i in range(len(origenes)):
    solicitudes=[]
    for j in range(len(destinos)):
        solicitud="https://maps.googleapis.com/maps/api/distancematrix/json?destinations="+destinos[j]+"&origins="+origenes[i]+"&key="+key_API_google
        solicitudes.append(solicitud)
    todas_solicitudes.append(solicitudes)

In [41]:
todas_solicitudes2 = pd.DataFrame(todas_solicitudes)

In [43]:
transposed_todas_solicitudes2 = todas_solicitudes2.transpose()

In [46]:
def solicitud_maps(url):
    try:
        payload = {}
        headers = {}
        response = requests.request("GET", url, headers=headers, data=payload)
        x = response.json()
        drive_time_seconds = x['rows'][0]['elements'][0]['duration']['value']
        distance_meters = x['rows'][0]['elements'][0]['distance']['value']
        return [drive_time_seconds, distance_meters]
    except KeyError:
        return 0

In [47]:
df["2"] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[0]), axis=1)
df["3"] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[1]), axis=1)
df["4"] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[2]), axis=1)
df["5"] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[3]), axis=1)
df["6"] = transposed_todas_solicitudes2.apply(lambda x: solicitud_maps(x[4]), axis=1)

In [49]:
df.to_csv("FINAL_tiempos_bomberos_saludnuevos.csv",index=True)

In [50]:
df2=df.copy(deep=True)

In [52]:
df2["2"] = df2["2"] .apply(lambda x: x[0])
df2["3"] = df2["3"] .apply(lambda x: x[0])
df2["4"] = df2["4"] .apply(lambda x: x[0])
df2["5"] = df2["5"] .apply(lambda x: x[0])
df2["6"] = df2["6"] .apply(lambda x: x[0])

In [53]:
df2.to_csv("TIEMPOS_VIAJE_SEG_CLUSTERS_BOMBEROS_NUEVOS.csv",index=True)